In [2]:
import numpy as np 
import sys 
import matplotlib.pyplot as plt
import time
import scipy.stats as st
import scipy.cluster.vq as cvq
import multiprocessing as mp
import image_tools as it
from scipy.ndimage import imread


import numpy.ma as ma
import scipy.stats as st
import scipy.signal as si
from astropy.io import fits
from astropy.table import Table
import scipy.ndimage.morphology as morph
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline
%reload_ext autoreload

import parameters
#import image_tools as it
import gibbs_sampler as gs
import fields_tools as ft
import seg_OTMF as sot

sys.path.insert(0,'./libext/MPDAF')
import mpdaf

from mpdaf.obj import WCS
from mpdaf.obj import WaveCoord
from mpdaf.obj import Image
from mpdaf.obj import Spectrum
from mpdaf.obj import Cube


from mpdaf.sdetect import Source, SourceList
from mpdaf.sdetect import Catalog

In [3]:
nom_cube = 'mosaic' # [mosaic] or udf

In [5]:
""" Catalog loading"""

if nom_cube=='mosaic':
    cat = Catalog.read('./data/mosaic_lya_sources/mosaic_c031_e028_withz.fits')

    cat_lya=cat[cat['TYPE']==6]

    print 'Total de %.0f LAE dans la mosaïque.'%len(cat_lya)
    
elif nom_cube=='udf10':

    cat = Catalog.read('./data/udf10_c031_e026_withz.vot')
    
    cat_lya=cat[cat['TYPE']==6]
    
    print 'Total de %.0f LAE dans le cube UDF10'%len(cat_lya)
    
else:
    print 'Erreur sur le nom de cube !'

Total de 745 LAE dans la mosaïque.


/home/miv/courbot/anaconda/lib/python2.7/site-packages/astropy/table/column.py:1095: MaskedArrayFutureWarning: setting an item on a masked array which has a shared mask will not copy the mask and also change the original mask array in the future.
Check the NumPy 1.11 release notes for more information.
  ma.MaskedArray.__setitem__(self, index, value)


In [ ]:
"""Object list"""

if nom_cube=='mosaic':
    liste_obj = (63,   68,   82,  102,  109,  126,  148,  153,  168,  171,  183, 214,  218,  262,  263,  298,
            299,  324,  364,  399,  400,  489, 579,  587,  590,  606,  619,  736, 1059, 1087, 1113, 1228,
            1445, 1446, 1477, 1478, 1486, 1581, 1583, 1593, 1665, 1670, 1711, 1714, 1723, 1724, 1726, 1737,
            1756, 1843, 1864, 1931, 1969, 1971, 1991, 2049, 2103, 2132, 2155, 2171, 2178, 2211, 2217, 2264,
            2333, 2335, 2363, 2389, 2403, 2467, 2495, 2515, 2522, 2547, 2598, 2634, 2672, 2687, 2727, 2733,
            2817, 2847, 2861, 2865, 2885, 3027, 3055, 3089, 3138, 3156, 3219, 3234, 3238, 3240, 3242, 3257,
            3286, 3305, 3329, 3335, 3421, 3426, 3480, 3484, 3492, 3494, 3585, 3589, 3611, 3634, 3672, 3723,
            3746, 3797, 3858, 3930, 3933, 3946, 4141, 4205, 4231, 4333, 4381, 4515, 4598, 4661, 4755, 4776,
            4802, 4842, 4870, 5027, 5225, 5281, 5308, 5439, 5452, 5566, 5949, 6101, 6341, 6372, 6375, 6376,
            6389, 6391, 6398, 6423, 6430, 6447, 6457, 6458, 6459, 6483, 6485, 6489, 6492, 6497, 6507, 6520,
            6537, 6550, 6565, 6570, 6572, 6578)

    print 'Total de %.0f objets retenus dans la mosaïque.'%len(liste_obj)
else:
    liste_obj = (78,149,153,157,183,208,214,237,547,619,802,828,6311,6312,6290,6295)
    
    print 'Total de %.0f objets retenus dans UDF10.'%len(liste_obj)

In [ ]:
"""Cube loading"""
if nom_cube=='mosaic':
    cube = Cube('./data/DATACUBE_UDF-MOSAIC.fits')
else:
    cube = Cube('./data/DATACUBE_UDF-10.fits')